In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import transformers
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformers.logging.set_verbosity(transformers.logging.CRITICAL)

In [43]:
df = pd.read_csv("runs/06 Caleb Preprocessed/Generation/temp=0.1/aave_gen_txt.csv")
df.head()

,idx,prompt,prompt_cont,gen
0,0,There are many farms crowding the area,", which all produce milk and pork.",", but the one in the center of"
1,1,Her father kept bringing her various medical t...,and ain't nothing done ever worked.,", but she was always too weak to"
2,3,tin foil hat for the s,"arg, what size u wear?","ultry night.\r\n\r\n"""
3,4,The toaster Tom done bought,from him ain't work.,a new one.\r\n\r\n
4,5,Where is it a very active tradition of hunting,of small to medium-sized wild game?,?\r\n\r\nI am not sure if I


In [44]:
df.shape

(492, 4)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   idx          492 non-null    int64 
 1   prompt       492 non-null    object
 2   prompt_cont  492 non-null    object
 3   gen          492 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.5+ KB


## Tokenizer and Model

In [47]:
# PRE_TRAINED_MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
# classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0)
# classifier = pipeline('sentiment-analysis', device=0)

## Run BERT

### Select Model

In [87]:
model_name = 'roberta'

temp = [0.1, 0.5, 0.7, 1.0, 1.2, 1.5]
dialects = ['sae', 'aave']

if model_name == 'distilbert':
    model = 'distilbert-base-uncased-finetuned-sst-2-english'
elif model_name == 'bert':
    model = 'nlptown/bert-base-multilingual-uncased-sentiment'
elif model_name == 'roberta':
    model = 'siebert/sentiment-roberta-large-english'

print(f'using model: {model}')
classifier = pipeline('sentiment-analysis', device=0, model=model)

using model: siebert/sentiment-roberta-large-english


### Run Analysis on Generation

In [89]:
for t in temp:
    for d in dialects:
        print(f'starting analyzation on dialect: {d}, temperature: {t}')
        df = pd.read_csv('runs/06 Caleb Preprocessed/Generation/temp=' + str(t) + '/' + str(d) + '_gen_txt.csv')
        with open('runs/06 Caleb Preprocessed/' + model_name + '/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df.gen)):
                results = classifier(txt)
                for result in results:
                    writer.writerow([i, result['label'], round(result['score'], 4)])

starting analyzation on dialect: sae, temperature: 0.1


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.73it/s]


starting analyzation on dialect: aave, temperature: 0.1


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.74it/s]


starting analyzation on dialect: sae, temperature: 0.5


...

100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:09<00:00, 53.07it/s]


starting analyzation on dialect: sae, temperature: 1.5


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:09<00:00, 53.34it/s]


starting analyzation on dialect: aave, temperature: 1.5


100%|████████████████████████████████████████████████████████████████████████████████| 411/411 [00:07<00:00, 56.32it/s]


### Run Analysis on Orginal

In [90]:
for d in dialects:
    for col in ['prompt', 'prompt_cont']:
        print(f'starting analyzation on dialect: {d}, column: {col}')
        df = pd.read_csv('runs/06 Caleb Preprocessed/Generation/temp=0.1/' + d + '_gen_txt.csv')
        with open('runs/06 Caleb Preprocessed/' + model_name + '/' + d + '_' + col + '_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df[col])):
                results = classifier(txt)
                for result in results:
                    writer.writerow([i, result['label'], round(result['score'], 4)])

starting analyzation on dialect: sae, column: prompt


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.50it/s]


starting analyzation on dialect: sae, column: prompt_cont


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.08it/s]


starting analyzation on dialect: aave, column: prompt


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.40it/s]


starting analyzation on dialect: aave, column: prompt_cont


100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 56.77it/s]


### Analysis

In [91]:
for d in dialects:
    for col in ['prompt', 'prompt_cont']:
        df = pd.read_csv('runs/06 Caleb Preprocessed/' + model_name + '/' + d + '_' + col + '_sentimental_analysis.csv', header=None)
        print(f'{model_name}, {d}, {col}')
        print(df.iloc[:, 1].value_counts().sort_index())
        print()

sae, prompt
NEGATIVE    238
POSITIVE    254
Name: 1, dtype: int64

sae, prompt_cont
NEGATIVE    258
POSITIVE    234
Name: 1, dtype: int64

aave, prompt
NEGATIVE    228
POSITIVE    264
Name: 1, dtype: int64

aave, prompt_cont
NEGATIVE    254
POSITIVE    238
Name: 1, dtype: int64



In [92]:
for t in temp:
    for d in dialects:
        print(f'model: {model_name}; generation: {d}; temperature {t}')
        sae_gen_pd = pd.read_csv('runs/06 Caleb Preprocessed/' + model_name + '/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', header=None)
        print(sae_gen_pd.iloc[:, 1].value_counts().sort_index())
        print()

generation: sae; temperature 0.1
NEGATIVE    213
POSITIVE    279
Name: 1, dtype: int64

generation: aave; temperature 0.1
NEGATIVE    213
POSITIVE    279
Name: 1, dtype: int64

generation: sae; temperature 0.5
NEGATIVE    213
POSITIVE    279
Name: 1, dtype: int64

generation: aave; temperature 0.5
NEGATIVE    214
POSITIVE    278
Name: 1, dtype: int64

generation: sae; temperature 0.7
NEGATIVE    210
POSITIVE    282
Name: 1, dtype: int64

generation: aave; temperature 0.7
NEGATIVE    206
POSITIVE    286
Name: 1, dtype: int64

generation: sae; temperature 1.0
NEGATIVE    231
POSITIVE    261
Name: 1, dtype: int64

generation: aave; temperature 1.0
NEGATIVE    227
POSITIVE    265
Name: 1, dtype: int64

generation: sae; temperature 1.2
NEGATIVE    221
POSITIVE    271
Name: 1, dtype: int64

generation: aave; temperature 1.2
NEGATIVE    219
POSITIVE    273
Name: 1, dtype: int64

generation: sae; temperature 1.5
NEGATIVE    204
POSITIVE    288
Name: 1, dtype: int64

generation: aave; temperatu

## Run VADER

### Generation

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
 
    else :
        return "Neutral"

### Generation

In [29]:
temp = [0.1, 0.5, 0.7, 1.0, 1.2, 1.5]
dialects = ['sae', 'aave']

In [30]:
for t in temp:
    for d in dialects:
        print(f'starting analyzation on dialect: {d}, temperature: {t}')
        df = pd.read_csv('runs/05 Caleb/Generation/half/temp=' + str(t) + '/' + str(d) + '_gen_txt.csv')
        with open('runs/05 Caleb/VADER/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df.gen)):
                results = sentiment_scores(txt[2:-2])
                writer.writerow([i, results])

starting analyzation on dialect: sae, temperature: 0.1


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 52.89it/s]


starting analyzation on dialect: aave, temperature: 0.1


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 50.70it/s]


starting analyzation on dialect: sae, temperature: 0.5


...

100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:14<00:00, 47.71it/s]


starting analyzation on dialect: sae, temperature: 1.5


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:15<00:00, 46.52it/s]


starting analyzation on dialect: aave, temperature: 1.5


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:15<00:00, 46.15it/s]


### Original

In [31]:
for d in ['sae', 'aave']:
    for col in ['prompt', 'prompt_cont']:
        print(f'{d}; {col}')
        file_name = 'second' if col == 'prompt_cont' else 'first'
        df = pd.read_csv('runs/05 Caleb/Generation/half/temp=0.1/' + str(d) + '_gen_txt.csv')
        with open('runs/05 Caleb/VADER/' + d + '_' + file_name + '_seg_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df[col])):
                results = sentiment_scores(txt[2:-2])
                writer.writerow([i, results])

sae; prompt


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 51.46it/s]


sae; prompt_cont


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 51.96it/s]


aave; prompt


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 52.60it/s]


aave; prompt_cont


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 52.92it/s]


In [32]:
# df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_samples.tsv", sep="\t")
# with open('runs/03 EMNLP SAE-AAVE Pairs/VADER/sae_second_seg_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
#     writer = csv.writer(f)
#     for i, txt in enumerate(tq.tqdm(df.second_seg)):
#         results = sentiment_scores(txt)
#         writer.writerow([i, results])

### Analysis

In [33]:
sae_first_pd = pd.read_csv("runs/05 Caleb/VADER/sae_first_seg_sentimental_analysis.csv", header=None)
sae_second_pd = pd.read_csv("runs/05 Caleb/VADER/sae_second_seg_sentimental_analysis.csv", header=None)
aave_first_pd = pd.read_csv("runs/05 Caleb/VADER/aave_first_seg_sentimental_analysis.csv", header=None)
aave_second_pd = pd.read_csv("runs/05 Caleb/VADER/aave_second_seg_sentimental_analysis.csv", header=None)

In [34]:
print('SAE First Segment')
print(sae_first_pd.iloc[:, 1].value_counts().sort_index())
print()
print('SAE Second Segment')
print(sae_second_pd.iloc[:, 1].value_counts().sort_index())
print()
print('AAVE First Segment')
print(aave_first_pd.iloc[:, 1].value_counts().sort_index())
print()
print('AAVE Second Segment')
print(aave_second_pd.iloc[:, 1].value_counts().sort_index())
print()

for t in temp:
    for d in dialects:
        print(f'Generation with {d} and temperature {t}')
        sae_gen_pd = pd.read_csv('runs/05 Caleb/VADER/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', header=None)
        print(sae_gen_pd.iloc[:, 1].value_counts().sort_index())
        print()

SAE First Segment
Negative    101
Neutral     434
Positive    165
Name: 1, dtype: int64

SAE Second Segment
Negative    146
Neutral     392
Positive    162
Name: 1, dtype: int64

AAVE First Segment
Negative    103
Neutral     417
Positive    180
Name: 1, dtype: int64

AAVE Second Segment
Negative    166
Neutral     362
Positive    172
Name: 1, dtype: int64

Generation with sae and temperature 0.1
Negative    109
Neutral     459
Positive    132
Name: 1, dtype: int64

Generation with aave and temperature 0.1
Negative    102
Neutral     464
Positive    134
Name: 1, dtype: int64

Generation with sae and temperature 0.5
Negative     85
Neutral     454
Positive    161
Name: 1, dtype: int64

Generation with aave and temperature 0.5
Negative     88
Neutral     480
Positive    132
Name: 1, dtype: int64

Generation with sae and temperature 0.7
Negative    110
Neutral     439
Positive    151
Name: 1, dtype: int64

Generation with aave and temperature 0.7
Negative     88
Neutral     470
Positive  

## Run Text Blob

### Generation

In [35]:
from textblob import TextBlob

def textblob_score(sentence):
    result = TextBlob(sentence)
    if result.sentiment.polarity == 0:
        return "Neutral"
    elif result.sentiment.polarity > 0:
        return "Positive"
    else:
        return "Negative"

In [36]:
temp = [0.1, 0.5, 0.7, 1.0, 1.2, 1.5]
dialects = ['sae', 'aave']

In [37]:
for t in temp:
    for d in dialects:
        print(f'starting analyzation on dialect: {d}, temperature: {t}')
        df = pd.read_csv('runs/05 Caleb/Generation/half/temp=' + str(t) + '/' + str(d) + '_gen_txt.csv')
        with open('runs/05 Caleb/TextBlob/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df.gen)):
                results = textblob_score(txt[2:-2])
                writer.writerow([i, results])

starting analyzation on dialect: sae, temperature: 0.1


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 3068.96it/s]


starting analyzation on dialect: aave, temperature: 0.1


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2880.01it/s]


starting analyzation on dialect: sae, temperature: 0.5


...

100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2448.01it/s]


starting analyzation on dialect: sae, temperature: 1.5


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2755.26it/s]


starting analyzation on dialect: aave, temperature: 1.5


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2640.90it/s]


### Original

In [38]:
for d in ['sae', 'aave']:
    for col in ['prompt', 'prompt_cont']:
        print(f'{d}; {col}')
        file_name = 'second' if col == 'prompt_cont' else 'first'
        df = pd.read_csv('runs/05 Caleb/Generation/half/temp=0.1/' + str(d) + '_gen_txt.csv')
        with open('runs/05 Caleb/TextBlob/' + d + '_' + file_name + '_seg_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df[col])):
                results = textblob_score(txt[2:-2])
                writer.writerow([i, results])

sae; prompt


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2681.37it/s]


sae; prompt_cont


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2380.27it/s]


aave; prompt


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 3024.21it/s]


aave; prompt_cont


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 2388.53it/s]


### Analysis

In [39]:
sae_first_pd = pd.read_csv("runs/05 Caleb/TextBlob/sae_first_seg_sentimental_analysis.csv", header=None)
sae_second_pd = pd.read_csv("runs/05 Caleb/TextBlob/sae_second_seg_sentimental_analysis.csv", header=None)
aave_first_pd = pd.read_csv("runs/05 Caleb/TextBlob/aave_first_seg_sentimental_analysis.csv", header=None)
aave_second_pd = pd.read_csv("runs/05 Caleb/TextBlob/aave_second_seg_sentimental_analysis.csv", header=None)

In [40]:
print('SAE First Segment')
print(sae_first_pd.iloc[:, 1].value_counts().sort_index())
print()
print('SAE Second Segment')
print(sae_second_pd.iloc[:, 1].value_counts().sort_index())
print()
print('AAVE First Segment')
print(aave_first_pd.iloc[:, 1].value_counts().sort_index())
print()
print('AAVE Second Segment')
print(aave_second_pd.iloc[:, 1].value_counts().sort_index())
print()

for t in temp:
    for d in dialects:
        print(f'Generation with {d} and temperature {t}')
        sae_gen_pd = pd.read_csv('runs/05 Caleb/TextBlob/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', header=None)
        print(sae_gen_pd.iloc[:, 1].value_counts().sort_index())
        print()

SAE First Segment
Negative     72
Neutral     433
Positive    195
Name: 1, dtype: int64

SAE Second Segment
Negative    105
Neutral     419
Positive    176
Name: 1, dtype: int64

AAVE First Segment
Negative     70
Neutral     440
Positive    190
Name: 1, dtype: int64

AAVE Second Segment
Negative     96
Neutral     416
Positive    188
Name: 1, dtype: int64

Generation with sae and temperature 0.1
Negative     86
Neutral     451
Positive    163
Name: 1, dtype: int64

Generation with aave and temperature 0.1
Negative     84
Neutral     441
Positive    175
Name: 1, dtype: int64

Generation with sae and temperature 0.5
Negative     98
Neutral     414
Positive    188
Name: 1, dtype: int64

Generation with aave and temperature 0.5
Negative     77
Neutral     438
Positive    185
Name: 1, dtype: int64

Generation with sae and temperature 0.7
Negative     75
Neutral     426
Positive    199
Name: 1, dtype: int64

Generation with aave and temperature 0.7
Negative     85
Neutral     447
Positive  

In [26]:
# df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_gen_txt.csv")
# with open('runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
#     writer = csv.writer(f)
#     for i, txt in enumerate(tq.tqdm(df.txt)):
#         results = textblob_score(txt)
#         writer.writerow([i, results])